In [1]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.optimizers import SGD

import numpy as np
import pickle

In [2]:
(X, Y), (X_test, Y_test) = pickle.load(open("data.p", "rb"))

In [3]:
OUTPUT = 30

network = input_data(shape=[None, 96, 96, 1])
network = fully_connected(network, OUTPUT, activation='linear', weights_init=tflearn.initializations.zeros())

In [4]:
sgd = SGD(learning_rate=0.01, lr_decay=0.96, decay_step=100)

In [5]:
regression_ = regression(network, optimizer=sgd, loss='mean_square')

In [6]:
model = tflearn.DNN(regression_, checkpoint_path='checkpoints/', max_checkpoints=3, tensorboard_verbose=0, tensorboard_dir='/tmp/tflearn_logs/')

INFO:tensorflow:Summary name Loss/ (raw) is illegal; using Loss/__raw_ instead.


In [7]:
model.fit(X, Y, run_id='v1', n_epoch=50, validation_set=0.1, shuffle=True, show_metric=True, batch_size=64, snapshot_step=200, snapshot_epoch=False)

Training Step: 1550  | total loss: 0.00662 | time: 0.256s
| SGD | epoch: 050 | loss: 0.00662 - acc: 0.7115 -- iter: 1926/1926
